In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

import requests

from io import StringIO, BytesIO

%matplotlib inline

# Singular Value Decomposition (SVD)

Sejam $m$ e $n$ as dimensões de uma matriz  $A \in \mathbb{R}^{m \times n}$, então a decomposição em valores singulares de $A$ é dada na forma abaixo.

\begin{equation}
    A = U \Sigma V^T
\end{equation}
    
- $U \in \mathbb{R}^{m \times m}$ e $V \in \mathbb{R}^{n \times n}$ são matrizes ortogonais de vetores singulares de $A$
- $\Sigma = \mbox{diag}(\sigma_1, \ldots, \sigma_r)$ é matriz diagonal
- $\sigma_1 \geq \sigma_2 \ldots \geq \sigma_r \geq 0$ são os valores singulares de $A$

In [ ]:
def dim(img):
    row, col, _ = img.shape
    print("Pixels: {} * {}".format(row, col))
    
def rgb(img):
    return img[:,:, 0], img[:,:, 1], img[:,:, 2]

def mult(U, S, V):
	return np.dot(U, np.dot(np.diag(S), V))

In [ ]:
def compress(img, k):
    
    row, col, _ = img.shape
    
    # Separacao dos canais RGB
    img_r = img[:,:, 0]
    img_g = img[:,:, 1] 
    img_b = img[:,:, 2]
    
    U_r, S_r, V_r = np.linalg.svd(img_r, full_matrices=True)
    U_g, S_g, V_g = np.linalg.svd(img_g, full_matrices=True)
    U_b, S_b, V_b = np.linalg.svd(img_b, full_matrices=True)
    
    # Selecao dos k-primeiros valores singulares 
    U_r_k = U_r[:, 0:k]
    V_r_k = V_r[0:k, :]
    U_g_k = U_g[:, 0:k]
    V_g_k = V_g[0:k, :]
    U_b_k = U_b[:, 0:k]
    V_b_k = V_b[0:k, :]

    S_r_k = S_r[0:k]
    S_g_k = S_g[0:k]
    S_b_k = S_b[0:k]
       
    # Reconstrucao usando uma aproximacao k-rank da imagem
    img_r_k = np.dot(U_r_k, np.dot(np.diag(S_r_k), V_r_k))
    img_g_k = np.dot(U_g_k, np.dot(np.diag(S_g_k), V_g_k))
    img_b_k = np.dot(U_b_k, np.dot(np.diag(S_b_k), V_b_k))
    
    img_reconst = np.zeros((row, col, 3))
    img_reconst[:, :, 0] = img_r_k
    img_reconst[:, :, 1] = img_g_k
    img_reconst[:, :, 2] = img_b_k
    
    img_reconst[img_reconst < 0] = 0
    img_reconst[img_reconst > 1] = 1
    
    original_bytes = img.nbytes
    #print("The space (in bytes) needed to store this image is {}".format(original_bytes))

    bytes_to_be_stored = sum([m.nbytes for m in [U_r, S_r, V_r , U_g, S_g, V_g , U_b, S_b, V_b]])
    #print("The space (in bytes) needed to store all matrices is {}".format(bytes_to_be_stored))

    compressed_bytes = sum([m.nbytes for m in [U_r_k, V_r_k, U_g_k, V_g_k, U_b_k, V_b_k, S_r_k, S_g_k, S_b_k]])
    #print("The space (in bytes) needed to store compressed matrices is {}".format(compressed_bytes))

    ratio = 100 * (compressed_bytes / original_bytes)
    #print("The compression ratio is {}%".format(ratio))
    
    return img_reconst, ratio

## Imagem

In [ ]:
r = requests.get('https://upload.wikimedia.org/wikipedia/commons/7/78/New_York_%2833559074896%29.jpg')

## Preto-e-branco

In [ ]:
from PIL import Image
img = Image.open(BytesIO(r.content))

In [ ]:
imggray = img.convert('LA')

imgmat = np.array(list(imggray.getdata(band=0)), float)
imgmat.shape = (imggray.size[1], imggray.size[0])
imgmat = np.matrix(imgmat)

plt.imshow(imgmat, cmap='gray')

In [ ]:
U, S, V = np.linalg.svd(imgmat)

fig = plt.figure()
    
for i in range(100, 1601, 100):
    
    reconstimg = np.matrix(U[:, :i]) * np.diag(S[:i]) * np.matrix(V[:i, :])   
    plt.imshow(reconstimg, cmap='gray')
    #plt.title("n = {}".format(i))
    #fig.savefig("{}.png".format(i))    

In [ ]:
fig = plt.figure()

plt.plot(np.diag(S), color='darkblue')
#plt.semilogx(np.diag(S), basex=10, color='darkblue', linewidth = 0.5)
plt.yscale('log', nonposy='clip')
plt.title('Valores singulares (log 10)')
plt.show()
#fig.savefig('svdlog10.png')

## Colorida

In [ ]:
array = np.array(img)
array = array / 255

dim(array)

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(1, 1, 1)

imgpplot = plt.imshow(array)
    
plt.title("original")
fig.savefig("data/original.png".format(i))

In [ ]:
for i in range(100, 1001, 100):
    
    fig = plt.figure(figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(1, 1, 1)
    
    reconst, ratio = compress(array, i)    
    imgpplot = plt.imshow(reconst)
    
    plt.title("k = {}, ratio = {:.02f}%".format(i, ratio))
    fig.savefig("data/{}.png".format(i))

In [ ]:
import os
import glob

# filepaths
fp_in = "data/*.png"
fp_out = "data/image.gif"

# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
imgs = [Image.open(f).copy() for f in sorted(glob.glob(fp_in), key=os.path.getmtime)]

imgs[0].save(fp=fp_out, format='GIF', append_images=imgs[1:], save_all=True, duration=2000, loop=10)